In [2]:

import math, re, ccxt
from pprint import pformat

def get_data_for_graph(price1 = "last", price2 = "last"):
    print('data for: ', price1, price2)
    wex = ccxt.wex()
    data_for_graph = {}
    data = wex.fetch_tickers()
    for key in data:
        data_for_graph.update({key: data[key][price1]})
    ticker_list = list(data_for_graph.keys())
    ticker_list2 = []
    for key in ticker_list:
        ticker_list2.append("{1}/{0}".format(*key.split('/'))) #обратные тикеры
    
    for key in range(0, len(ticker_list)):
        #обратные котировки
        data_for_graph.update({ticker_list2[key]: 1 / (data[ticker_list[key]][price2])})
    
    print(pformat(data_for_graph))
    return data_for_graph
  


In [3]:
'''

Создаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, 
котировки переводим в логарифмический вид, чтобы можо было складывать вместо 
умножения
'''


'\n\nСоздаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, \nкотировки переводим в логарифмический вид, чтобы можо было складывать вместо \nумножения\n'

In [3]:
def create_graph(data_for_graph):
    graph = {}
    jsrates = data_for_graph
    
    pattern = re.compile("([A-Z]{3,5})/([A-Z]{3,5})")
    
    try:
        for key in jsrates:
            matches = pattern.match(key)
    
            conversion_rate = -math.log(float(jsrates[key]))
            
            from_rate = matches.group(1).encode('ascii', 'ignore')
    
            to_rate = matches.group(2).encode('ascii', 'ignore')
    
            if from_rate != to_rate:
                if from_rate not in graph:
                    graph[from_rate] = {}
                graph[from_rate][to_rate] = float(conversion_rate)
    
    except AttributeError:
        print('Strange rate')
        pass
    
    #print('graph: ', '\n',pformat(graph))
    # воплощение алгоритма отсюда
    #  https://gist.github.com/joninvski/701720
    return graph


In [13]:



# Step 1: For each node prepare the destination and predecessor
def initialize(graph, source):
    d = {}  # Stands for destination
    p = {}  # Stands for predecessor
    for node in graph:
        d[node] = float('Inf')  # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0  # For the source we know how to reach
    return d, p


def relax(node, neighbour, graph, d, p):
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]:
        # Record this lower distance
        d[neighbour] = d[node] + graph[node][neighbour]
        p[neighbour] = node


def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    #print('next node:', next_node)
    while True:
        
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph)-1):  # Run this until is converges
        for u in graph:
            for v in graph[u]:  # For each neighbour of u
                relax(u, v, graph, d, p)  # Lets relax it

    # Step 3: check for negative-weight cycles
    for u in graph:
        #print('u: ', u)
        for v in graph[u]: #graph[u]
            
            if d[v] > d[u] + graph[u][v]:                
                return (retrace_negative_loop(p, source))
    return None


In [20]:
data_for_graph = get_data_for_graph(price1='bid', price2='ask')


data for:  bid ask


{'BCH/BCHET': 1.1025358324145533,
 'BCH/BTC': 0.0912,
 'BCH/DASH': 2.019,
 'BCH/ETH': 1.477,
 'BCH/EUR': 596.434,
 'BCH/LTC': 5.789,
 'BCH/RUR': 41756.4,
 'BCH/USD': 722.918,
 'BCH/ZEC': 3.274,
 'BCHET/BCH': 0.9,
 'BTC/BCH': 10.905125408942203,
 'BTC/BTCET': 1.2224938875305624,
 'BTC/DASH': 22.12878955521133,
 'BTC/ETH': 16.105653084232564,
 'BTC/EUR': 6501.80782,
 'BTC/LTC': 63.051702395964696,
 'BTC/NMC': 4761.9047619047615,
 'BTC/NVC': 2173.913043478261,
 'BTC/PPC': 4166.666666666667,
 'BTC/RUR': 456100.0,
 'BTC/USD': 7925.0,
 'BTC/ZEC': 35.842293906810035,
 'BTCET/BTC': 0.812,
 'DASH/BCH': 0.4921259842519685,
 'DASH/BTC': 0.04497,
 'DASH/DSHET': 1.1947431302270013,
 'DASH/ETH': 0.726,
 'DASH/EUR': 293.988,
 'DASH/LTC': 2.85,
 'DASH/RUR': 20505.215,
 'DASH/USD': 355.95682,
 'DASH/ZEC': 1.612,
 'DSHET/DASH': 0.835,
 'ETH/BCH': 0.6738544474393531,
 'ETH/BTC': 0.06166,
 'ETH/DASH': 1.36986301369863,
 'ETH/ETHET': 1.2062726176115803,
 'ETH/EUR': 402.06204,
 'ETH/LTC': 3.90241,
 'ETH/RUR

In [21]:
paths = []

graph = create_graph(data_for_graph)
for key in graph:
    
    path = bellman_ford(graph, key)
    
    if path not in paths and not None:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        print('cycle lenght:', len(path))
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(-graph[start][end])
                money *=rate
                #money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")

 

No opportunity here :(


